# Import packages

In [ ]:
import pandas as pd
import datetime

from statsmodels.formula.api import ols

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from utils import diff_in_diff_regr

# Parameters

In [ ]:
treatment_date = datetime.date(2022, 5, 31)

# Import data

In [ ]:
consumption_df = pd.read_csv('data/dummy_peer_consumption_with_pv.csv', index_col=0)

In [ ]:
consumption_df

# Test custom function

In [ ]:
ols_model = diff_in_diff_regr(consumption_df, treatment_date)

In [ ]:
ols_model.params['treatment']

# Data processing

In [ ]:
consumption_df.index = pd.DatetimeIndex(consumption_df.index)

In [ ]:
consumption_df = pd.melt(consumption_df, ignore_index=False, var_name='type')

In [ ]:
consumption_df

In [ ]:
consumption_df['post_treatment_period'] = (consumption_df.index.date > treatment_date).astype(int)

In [ ]:
consumption_df

In [ ]:
# Add treatment indicator
consumption_df['treatment'] = ((consumption_df['post_treatment_period']==1) &
                               (consumption_df['type']=='your consumption')).astype(int)

In [ ]:
# Make dummy variable out of type
consumption_df['type'] = (consumption_df['type']=='your consumption').astype(int)

In [ ]:
consumption_df

# Regression

In [ ]:
model = ols('value ~ type + post_treatment_period + treatment', data=consumption_df).fit(cov_type='HC1',)

In [ ]:
# Inspect results
print(model.summary())

# extract treatment effect
print(model.params['treatment'])

In [ ]:
type(model.conf_int(alpha=0.05)) #['treatment']

In [ ]:
model.conf_int(alpha=0.05).loc['treatment']

In [ ]:
model.conf_int(alpha=0.05).loc['treatment'][0]

# Visualize results

In [ ]:
type(model)

In [ ]:
est = model.params['treatment']
errors = [[est - model.conf_int(alpha=0.05).loc['treatment'][0]],
[model.conf_int(alpha=0.05).loc['treatment'][1] - est]]

In [ ]:
errors

In [ ]:
plt.bar([0], est, yerr=errors, color=['lightgreen'])

plt.ylabel('kWh')
plt.title('Estimated monthly effect of \n PV installation on electricity consumption')
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
plt.show()